In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Direct Feature Engineering

In [11]:
hockey_train = pd.read_csv('./data/clean/train.csv'
                     ,encoding = "ISO-8859-1")
hockey_test = pd.read_csv('./data/clean/test.csv'
                          ,encoding = "ISO-8859-1")
hockey_test_y = pd.read_csv('./data/clean/test_salaries.csv'
                          ,encoding = "ISO-8859-1")

In [12]:
def combine_train_and_test(train_df, test_df, test_response):
    test_df = pd.concat([test_df, test_response], axis = 1)
    return pd.concat([train_df, test_df],ignore_index = True, sort = False)

In [13]:
hockey = combine_train_and_test(hockey_train, hockey_test, hockey_test_y)

In [14]:
hockey['Age'] = 117 - pd.to_numeric(hockey['Born'].str[0:2])
# hockey = hockey.drop(['Born', 'City', 'Last Name', 'First Name', 'Cntry'], axis = 1)

def nationality_group(df, nationalityCol):
    # A function to feature engineering the 'Nationality column'
    # Changes it from 16 unique values to 5 to prevent overfitting
    scandanavianNations = ['SWE','NOR','FIN']
    otherNations = ['CHE','CZE','FRA','DEU','SVK','AUT','DNK','LVA','HRV','GBR']
    df.loc[(df[nationalityCol].isin(scandanavianNations)), nationalityCol] = 'Scandanavian'
    df.loc[(df[nationalityCol].isin(otherNations)), nationalityCol] = 'Other'
    return df
hockey = nationality_group(hockey, 'Nat')

In [15]:
# Code used to group and remove provinces and states that are only seen a few times
# Useful to prevent overfitting
prs = hockey.groupby('Pr/St').agg({'Pr/St':['count']}).reset_index()
prs.columns = ['pr/st','count']
extreneousStates = list(prs.loc[(prs['count'] < 10)]['pr/st'])
hockey.loc[(hockey['Pr/St'].isin(extreneousStates)),'Pr/St'] = 'Other'

# Save Processed Data to be used by Model Pipeline

In [27]:
y = hockey['Salary']
X = hockey.drop('Salary', axis = 1)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [29]:
X_train.to_csv('./data/processed/X_train.csv', index= False)
X_test.to_csv('./data/processed/X_test.csv', index= False)
y_train.to_csv('./data/processed/y_train.csv', index= False)
y_test.to_csv('./data/processed/y_test.csv', index= False)

/Users/nickparker/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/nickparker/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.
